In [19]:
import pandas as pd
import numpy as np
from shutil import copy, rmtree 
import os
from sklearn.model_selection import train_test_split

In [20]:
data_pd = pd.read_csv(r'C:\Users\user\OneDrive\documents\code\Python\Projects\deep_learning_skin\resource\HAM10000\HAM10000_metadata.csv')
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [21]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,dx,dx_type,age,sex,localization,dataset
lesion_id,,,,,,,
HAM_0000000,2,2,2,2,2,2,2
HAM_0000001,1,1,1,1,1,1,1
HAM_0000002,3,3,3,3,3,3,3
HAM_0000003,1,1,1,1,1,1,1
HAM_0000004,1,1,1,1,1,1,1


In [22]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [23]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'

In [24]:
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicates


In [25]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

In [26]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

In [27]:
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,duplicates,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,duplicates,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,duplicates,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,duplicates,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,duplicates,train


In [28]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,is_duplicate
4535,HAM_0003306,ISIC_0025259,nv,follow_up,45.0,female,trunk,vidir_molemax,no
5789,HAM_0002543,ISIC_0025645,nv,follow_up,45.0,male,abdomen,vidir_molemax,no
3142,HAM_0003349,ISIC_0028917,nv,follow_up,65.0,female,lower extremity,vidir_molemax,no
3459,HAM_0002124,ISIC_0025175,nv,follow_up,50.0,female,upper extremity,vidir_molemax,no
2354,HAM_0006203,ISIC_0026456,vasc,histo,60.0,male,upper extremity,vidir_modern,no


In [29]:
len(train_df),len(test_df)#(9187, 828)

(9187, 828)

In [30]:
# Image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

# len(train_list),len(test_list)#(9187, 828)

#Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [31]:
#create store
train_dir = os.path.join(os.getcwd(), 'train_dir')
test_dir = os.path.join(os.getcwd(), 'test_dir')

In [32]:
os.mkdir(train_dir)
os.mkdir(test_dir)

In [33]:
# Image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [34]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [35]:
for i in targetnames:
    directory1=train_dir+'/'+i
    directory2=test_dir+'/'+i
    os.mkdir(directory1)
    os.mkdir(directory2)

In [40]:
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('Images', file_name)

    # copying the image from the source to target file
    target = os.path.join(train_dir, label, file_name)

    copy(source, target)

In [41]:
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('Images', file_name)
    
    # copying the image from the source to target file
    target = os.path.join(test_dir, label, file_name)

    copy(source, target)